In [1]:
# get libraries 

import numpy as np # to work with arrays
import random as rand # to work with initial conditions
import matplotlib.pyplot as plt # to display matrix  
from IPython.display import clear_output # to clear previous matrix display
import imageio # to work with gif
import itertools # to create matrix index list of lists
from PIL import Image # used to read images to list from directory
import glob 

In [2]:
# inital structure

arrays = [] # initialize list of arrays; elements to be displayed in sequence with matplot
no_arrays = 100 # number of arrays in list after initial state
array_dim = 100 # array element row/column dimensions; must be even 

# initial condition

# test01 all cells random 100 generations
# array0 = np.random.randint(2, size=(array_dim,array_dim) ) # random first array

# test02 1 row randomly chosen 100 generations
# array0 = np.zeros([array_dim, array_dim]) # default array with all zeros
# array0_row = np.random.randint(low=0, high=array_dim) # select a random row
# array0[array0_row] = [ 1 for x in range(0,array_dim) ] # write all values in randomly selected row to 1 

# test03 # all cells randomly chosen with probaility of 1 at 10% 100 generations
p1 = 0.1
array0 = np.random.choice([0, 1], p=[ 1 - p1, p1], size=(array_dim, array_dim))

In [3]:
# matrix adjacent index positions

# list of index positions for each row or column based on array_dim initial condition  
array_dim_list = [x for x in range(0,array_dim)] 

# list of tuples of (row,col) index positions for each cell in matrix
matrix_indices = list(itertools.product(array_dim_list, array_dim_list))

# create list of lists of the 8 adjacent index positions for each cell in matrix 
adj_idx_of_matrix_cell = []

for idx in matrix_indices:
    
    for _ in range(8):
        
        adj_idx = [ (idx[0]-1, idx[1]-1), (idx[0]-1, idx[1]+0), (idx[0]-1, idx[1]+1),
                    (idx[0]+0, idx[1]-1),                       (idx[0]+0, idx[1]+1),
                    (idx[0]+1, idx[1]-1), (idx[0]+1, idx[1]+0), (idx[0]+1, idx[1]+1) 
                  ]
                   
    adj_idx_of_matrix_cell.append(adj_idx) 

In [4]:
# function to create new generation

def create_new_generation(old_generation):
    
    # create a list of a count of neighbours adjacent to each cell in old generation

    neighbours_list = []

    for cell in adj_idx_of_matrix_cell:
    
        neighbours = 0
    
        for adj_idx in cell:
        
            # handle adjacent cells row/col index that are outside matrix index positions    
            # note that -1 index references refer to the end of element of list, so try/except did not work
            if adj_idx[0] < 0 or adj_idx[0] > (array_dim -1) or adj_idx[1] < 0 or adj_idx[1] > (array_dim -1):
                neighbours += 0
            
            else: 
                # accumulate the sum of the adjacent values retrieved from the input array 
                neighbours += old_generation[ adj_idx[0] ][ adj_idx[1] ]
            
        neighbours_list.append(neighbours)
    
    # determine whether new generation cells are alive or dead based on old generation neighbour counts
    
    old_generation_list = [ int(_) for _ in np.nditer(old_generation) ] # input array to list 
    new_generation_list = []
    
    for idx in range(0,array_dim**2):
        
        # any live cell with two or three live neighbours lives on to the next generation\
        if old_generation_list[idx] == 1 and neighbours_list[idx] in [2,3]:
            new_generation_list.append(1) 
            
        # any live cell with fewer than two live neighbours dies, as if by underpopulation
        # any live cell with more than three live neighbours dies, as if by overpopulation
        elif old_generation_list[idx] == 1 and neighbours_list[idx] not in [2,3]:
            new_generation_list.append(0) 
                
        # any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction
        elif old_generation_list[idx] == 0 and neighbours_list[idx] == 3:
            new_generation_list.append(1) 
                    
        else:
            new_generation_list.append(0)
              
    return np.reshape(new_generation_list, (array_dim,array_dim) )

In [5]:
# place generations in a list  

arrays.append(array0) # add first generation to list

# add subsequent generations to list
for idx in range(no_arrays):
    
    arrays.append( create_new_generation(arrays[idx]) )    

In [6]:
# save png images

save_results_to = '/Users/akwong/Desktop/Game_of_Life_Results/Test03/'

counter = 0

for array in arrays:    
    
    fig, ax = plt.subplots(figsize=(10,10))
    
    # note when displaying with matplot 0 is black and 1 is white, so 1-array to invert display colour
    ax.imshow( 1 - array, cmap=plt.cm.gray, interpolation='none')
    plt.axis('off')
    
    # add leading zeroes for image sequence number 
    img_no = ['0'] * ( len(str(no_arrays)) - len(str(counter)) )
    img_no.append(str(counter))
    img_no = ''.join( _ for _ in img_no )
    
    plt.savefig(save_results_to + "image{img_no}.png".format(img_no=img_no))
    
    counter += 1
    
    plt.close(fig) # close each plot after used to save memory  

In [7]:
# read png files to list 

images = []

for filename in glob.glob('/Users/akwong/Desktop/Game_of_Life_Results/Test03/*.png'): 
    im = Image.open(filename)
    images.append(im)

In [8]:
# create gif file

kargs = { 'duration': 0.5} # display image duration

for image in images:
    imageio.mimsave('/Users/akwong/Desktop/Game_of_Life_Results/Test03/game_of_life_03.gif', images, **kargs)